In [3]:
import os

# Set JAVA_HOME to the path where your Java is installed
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("CSV Import") \
    .getOrCreate()

# Read your CSV file
df = spark.read.csv("./data/data.csv", header=True, inferSchema=True)
df.show()

spark.stop()


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, max as spark_max, datediff, lit, count, sum as spark_sum
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans

# Create a SparkSession
spark = SparkSession.builder \
    .appName("CSV Import with Legacy Time Parser") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Read the CSV file; adjust the path if necessary
# Assume the CSV has at least these columns: CustomerID, InvoiceDate, InvoiceNo, Amount
df = spark.read.csv("./data/data.csv", header=True, inferSchema=True)

# Convert InvoiceDate to a date type (adjust the format string as needed)
df = df.withColumn("InvoiceDate", to_timestamp(col("InvoiceDate"), "M/d/yyyy H:mm"))

# Create a new column 'TotalAmount' by multiplying 'Quantity' and 'UnitPrice'
df = df.withColumn("TotalAmount", col("Quantity") * col("UnitPrice"))

# Determine the reference date as the maximum InvoiceDate in the data
max_date = df.select(spark_max("InvoiceDate").alias("max_date")).collect()[0]["max_date"]

# Aggregate data to compute RFM metrics for each customer
# - Recency: days since last purchase (difference between reference date and last purchase date)
# - Frequency: total number of invoices (or transactions)
# - Monetary: total spend (sum of Amount)
rfm_df = df.groupBy("CustomerID").agg(
    datediff(lit(max_date), spark_max("InvoiceDate")).alias("Recency"),
    count("InvoiceNo").alias("Frequency"),
    spark_sum("TotalAmount").alias("Monetary")
)

print("RFM Metrics:")
rfm_df.show(10)

# Assemble the RFM metrics into a feature vector
assembler = VectorAssembler(
    inputCols=["Recency", "Frequency", "Monetary"],
    outputCol="features_unscaled"
)
rfm_features = assembler.transform(rfm_df)

# Standardize features for better clustering performance (optional but recommended)
scaler = StandardScaler(
    inputCol="features_unscaled", 
    outputCol="features",
    withStd=True, 
    withMean=True
)
scaler_model = scaler.fit(rfm_features)
rfm_features = scaler_model.transform(rfm_features)

# Train a KMeans model on the features; here we choose k=4 clusters (adjust as needed)
k = 4
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=k, seed=42)
model = kmeans.fit(rfm_features)

# Add cluster assignments to the DataFrame
rfm_clustered = model.transform(rfm_features)

print("RFM Clustering Results:")
rfm_clustered.select("CustomerID", "Recency", "Frequency", "Monetary", "cluster").show(10)

# Stop the Spark session when done
spark.stop()


RFM Metrics:


+----------+-------+---------+------------------+
|CustomerID|Recency|Frequency|          Monetary|
+----------+-------+---------+------------------+
|     17420|     50|       30| 598.8299999999999|
|     16861|     59|        8|            151.65|
|     16503|    106|       86|           1421.43|
|     15727|     16|      302| 5178.959999999998|
|     17389|      0|      224|31300.079999999994|
|     15447|    330|        9|            155.17|
|     14450|    180|       40|            483.25|
|     13623|     30|       86|            672.44|
|     13285|     23|      187|2709.1199999999994|
|     16339|    284|       20|109.95000000000002|
+----------+-------+---------+------------------+
only showing top 10 rows



RFM Clustering Results:


+----------+-------+---------+------------------+-------+
|CustomerID|Recency|Frequency|          Monetary|cluster|
+----------+-------+---------+------------------+-------+
|     17420|     50|       30| 598.8299999999999|      0|
|     16861|     59|        8|            151.65|      0|
|     16503|    106|       86|           1421.43|      3|
|     15727|     16|      302| 5178.959999999998|      0|
|     17389|      0|      224|31300.079999999994|      0|
|     15447|    330|        9|            155.17|      2|
|     14450|    180|       40|            483.25|      3|
|     13623|     30|       86|            672.44|      0|
|     13285|     23|      187|2709.1199999999994|      0|
|     16339|    284|       20|109.95000000000002|      2|
+----------+-------+---------+------------------+-------+
only showing top 10 rows

